# Approach Plate Parsing
Parsing of approach plates to create table of RNAV alititude and visisiblity requirements as well as airport meta data.

In [17]:
from pypdf import PdfReader

reader = PdfReader('./approach_plates/05251R14.PDF')

print(len(reader.pages))

page = reader.pages[0]

text = page.extract_text()
print(text)

1
P
P
A532
14
5498 X 100
0.5% UPWINCHESTER, VIRGINIA
(OKV)AL-5251 (FAA)
WINCHESTER RGNL
WINCHESTER, VIRGINIA(OKV) WINCHESTER RGNLRNAV (GPS) RWY 14
RNAV (GPS) RWY 14CLNC DEL
126.15AWOS-3
124.85 120.45  306.925UNICOM
122.9755498
726
726
(CTAF)POTOMAC APP CON
ELEVApt ElevTDZERwy Idg 
144°70712
W14AWAAS
CH    APP CRS
726
3700 CLADD
LNAV MDA
CIRCLINGMSA RW14 25 NM 
4700
RW14(3.6)107°3700
144°(4.3) 3500
CLADD
324°144°4 NM
RW143700
2700BONCIEMNAR
3500 * LNAV only*
*
4.3 NM144°
HIRL Rwy 14-32REIL Rwy 14TDZE7262844
2639
DALPV DA
VNAVLNAV/334 (400-1) 1060-1181060-1 334 (400-1   )18
1140-1 414 (500-1) 1140-11818414 (500-1   )
1180-1
454 (500-1)1220-1
494 (500-1) 654 (700-1   )12 1480-2
754 (800-2   )1234976- 250 (300-   )34BIPME
2000RW142 NM toWUSBI
to RW141.2 NM
*1400RW143.9 NM toWOGDI
1.2 NM 0.8 NM 1.9 NM 2.2 NM 2.6 NM     MUMSY on V44 eastbound.Procedure NA for arrivals at
     SOGBE on V4 westbound.Procedure NA for arrivals atZENDIACELO
RW143.9 NM toWOGDI RW142 NM toWUSBIEMNAR
SOGBE(IAF)BONCI

In [16]:
import os
import fitz
count = 1
approach_plate_dir = "./approach_plates/"
for filename in os.listdir(approach_plate_dir):
    doc = fitz.open(approach_plate_dir + "/" + filename)
    out = open(filename[:-4] + ".txt", "wb")
    for page in doc:
        text = page.get_text(sort=True).encode("utf8")
        out.write(text)
        out.write(bytes((12,)))
    out.close()
    count += 1
    if count > 10:
        break

